This notebook evaluates performance gains of `CudaCosineGreedy` against `matchms`.

Performance depends heavily on used hardware, as well as how correlated the spectra are with each other, (i.e. on average, how many common pairs of peaks do spectra have). 

This specific notebook evaluates the performance given the following below arguments (feel free to change these):

In [1]:
# We can use pickled version for much faster loading times
spectra_file = 'GNPS-LIBRARY-default-filter-nmax-2048.pickle'

# We take a random sample of spectra from said file

# Minimum size:
chunk_sizes_min = 32

# Maximum size
chunk_sizes_max = 10_000

# how many points to evaluate (in logspace) between min and max
num_evals = 15

# max number of peaks to retain in any spectra - larger numbers are marginally more accurate, but much slower
n_max_peaks = 1024

# Match limit
match_limit = 2048

# tolerance
tolerance = 0.1

# optimal batch size is hardware-dependent, but usually the best number is the largest the hardware can handle (without an OOM error)
batch_size = 4096

# Hardware matters! These results are only repeatable using this GPU (shown as an output)
! nvidia-smi -L
! echo Number of CPU cores $(nproc)

GPU 0: NVIDIA GeForce RTX 4090 (UUID: GPU-377e4e40-ce36-d6dc-f5fe-c716f302a235)
Number of CPU cores 64


In [2]:
! pip uninstall cudams -q -y
! pip install -q --upgrade git+https://github.com/tornikeo/cudams@main

In [4]:
from cudams.utils import argbatch, Timer
from cudams.similarity.spectrum_similarity_functions import cosine_greedy_kernel
from pathlib import Path
from tqdm import tqdm
from numba import cuda
import numpy as np
import torch
import matplotlib.pyplot as plt

assert cuda.is_available()

/opt/conda/lib/python3.10/site-packages/sparsestack/StackedSparseArray.py:4: DeprecationWarning: Please use `get_index_dtype` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import get_index_dtype


In [5]:
import pickle
from cudams.utils import download
from joblib import Parallel, delayed
from matchms.filtering import default_filters, normalize_intensities, reduce_to_number_of_peaks
from matchms.importing import load_from_mgf
from cudams.utils import mute_stdout

spectra = pickle.load(open(download(spectra_file),'rb'))

/opt/conda/lib/python3.10/site-packages/cudams/utils.py:267: UserWarning: As of 2024, ALL_GNPS.pickle is a large file (1.76GB) make sure the machine can handle this
  warnings.warn(
100%|██████████████████████████████████████| 1.47G/1.47G [00:00<00:00, 849GB/s]
SHA256 hash of downloaded file: 473593b86b35de9bc90c216b70ca7f64628bf577cb4bc64afd1d4d0a533ab1f9
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
/tmp/ipykernel_383/2885043538.py:8: ResourceWarning: unclosed file <_io.BufferedReader name='/root/.cache/pooch/a77e59fe0d8571b2d91a56e12790b36a-ALL_GNPS.pickle'>
  spectra = pickle.load(open(download(spectra_file),'rb'))


In [8]:
from cudams.similarity import CudaCosineGreedy

np.random.seed(42)
chunk_sizes_cu = np.round(np.logspace(
    np.log2(chunk_sizes_min), 
    np.log2(chunk_sizes_max), 
    num=num_evals, 
    base=2, 
    endpoint=True)
).astype(int)

times_cu = []
pairs_cu = []

# Kernel compilation might take a bit of time initially
kernel = CudaCosineGreedy(batch_size=batch_size, 
                          n_max_peaks=n_max_peaks)

# To force CUDA to load code to GPU, we need to do warmup
kernel.matrix(spectra[:64], spectra[:64])

# We avoid parallel here, since we only have one GPU, after all.
for chunk_size in tqdm(chunk_sizes_cu):
    chunk_size = min(len(spectra), chunk_size) # We might run out of spectra
    references = spectra[:chunk_size]
    queries = references # Pairwise
    with Timer() as timer:
        kernel.matrix(references, queries)
    times_cu.append(timer.duration)
    pairs_cu.append(len(references) * len(queries)) # We've processed all pairs

100%|██████████| 15/15 [02:05<00:00,  8.34s/it]


In [10]:
import json, os

benchmark_summary = dict(
        pairs_cu=pairs_cu,
        times_cu=times_cu,
        device=torch.cuda.get_device_name(),
        nproc=os.cpu_count(),    
    )
print(json.dumps(benchmark_summary,indent=1))

{
 "pairs_cu": [
  1024,
  2500,
  6084,
  14884,
  36100,
  88209,
  215296,
  524176,
  1279161,
  3115225,
  7595536,
  18524416,
  45158400,
  110103049,
  268435456
 ],
 "times_cu": [
  0.7942757579730824,
  0.8001212710514665,
  0.8135092230513692,
  0.8842686890857294,
  0.8815657449886203,
  0.9123411850305274,
  0.8964556299615651,
  0.9994770409539342,
  1.1101172500057146,
  1.4267874190118164,
  2.239893207908608,
  7.677725055953488,
  13.924317113007419,
  26.042265221010894,
  65.73885893099941
 ],
 "device": "NVIDIA GeForce RTX 4090",
 "nproc": 255
}
